![](../images/bunker_neve.jpeg)
<br>
Photo: [*The Bunker*](https://www.thebunkerstudio.com/)

# Who Do You Sound Like?
### Notebook 2: Feature Engineering & Metric Comparisons
#### Adam Zucker
---

## Contents
- **Section 1:** Package and data imports
- **Section 2:** Comparing Spotify metrics with Librosa metrics
- **Section 3:** Engineering conversions between Librosa and Spotify
- **Section 4:** Data exports

---
### Section 1
#### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns

import os
import IPython.display as ipd

import spotipy as sp
import librosa as lib
import librosa.display as libd

---

**BELOW:** Importing cleaned Spotify song dataset, sourced from [Kaggle](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data.csv). Some brief descriptions of less tangible features, as defined by [Spotify](https://developer.spotify.com/documentation/web-api/reference/):
- **Acousticness:** A confidence measure from 0.0 to 1.0 of whether the track is acoustic.
- **Danceability:** How suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. 
- **Energy:** A measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
- **Instrumentalness:** Predicts whether a track contains vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
- **Liveness:** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
- **Loudness:** The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 dB.
- **Popularity:** The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g., the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.
- **Speechiness:** Detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g., talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
- **Valence:** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g., happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g., sad, depressed, angry).

---

In [ ]:
df = pd.read_csv('../data_clean/spotify_kg_master.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# Data by year is quite evenly distributed.
df['year'].value_counts(normalize=True)

---
---
### Section 2
#### Metric Comparison

In [ ]:
# Creating a small dataframe of songs I know well to test Spotify metrics against those generated by Librosa.
spotify_metrics_test_df = pd.concat((df[51917:51918], df[51440:51441], df[55871:55872], df[36982:36983],
                                     df[43245:43246], df[48257:48258], df[40585:40586], df[46528:46529],
                                     df[40245:40246], df[12541:12542]))

In [ ]:
spotify_metrics_test_df

---

#### Librosa

In [ ]:
# Defining global sample rate, frame size, and hop length for incoming audio
sr = 44100
frame = 2048
hop = 512

In [ ]:
# Defining song filepaths for testing.
avicii_wav = '../songs_test/avicii_levels_radio.wav'
black_keys_wav = '../songs_test/blackkeys_tightenup.wav'
busta_rhymes_wav = '../songs_test/bustaryhmes_woohah.wav'
dua_lipa_wav = '../songs_test/dualipa_levitating.wav'
james_blake_wav = '../songs_test/jamesblake_retrograde.wav'
joey_wav = '../songs_test/joeybad_papertrails.wav'
joji_attn_wav = '../songs_test/joji_attention.wav'
joji_slowdancing_wav = '../songs_test/joji_slowdancing.wav'
kaleo_wav = '../songs_test/kaleo_waydownwego.wav'
radiohead_wav = '../songs_test/radiohead_lotusflower.wav'

In [ ]:
# ipd.Audio(dua_lipa_wav)

In [ ]:
# ipd.Audio(joji_attn_wav)

In [ ]:
# ipd.Audio(james_blake_wav)

**BELOW:** Loading songs into Librosa for feature extraction.

In [ ]:
avicii, sr = lib.load(avicii_wav, sr=sr)
black_keys, _ = lib.load(black_keys_wav, sr=sr)
busta_rhymes, _ = lib.load(busta_rhymes_wav, sr=sr)
dua_lipa, _ = lib.load(dua_lipa_wav, sr=sr)
james_blake, _ = lib.load(james_blake_wav, sr=sr)
joey, _ = lib.load(joey_wav, sr=sr)
joji_attn, _ = lib.load(joji_attn_wav, sr=sr)
joji_slowdancing, _ = lib.load(joji_slowdancing_wav, sr=sr)
kaleo, _ = lib.load(kaleo_wav, sr=sr)
radiohead, _ = lib.load(radiohead_wav, sr=sr)

In [ ]:
avicii.shape

**BELOW:** Based on a comparison of three songs, the duration metric matches up for Spotify and Librosa. This makes sense, as they're just measuring duration in seconds.

In [ ]:
# Measuring duration in seconds for Avicii - Levels using Librosa
print(f'Librosa measured duration in seconds (Avicii - Levels): {lib.get_duration(avicii, sr=sr, n_fft=frame, hop_length=hop)}')

# Comparing to Spotify measured duration
print(f"Spotify measured duration in seconds (Avicii - Levels): {df['duration_s'][40245]}")
print('-------------------------------------------------------------')
print('')

# Measuring duration in seconds for Dua Lipa - Levitating using Librosa
print(f'Librosa measured duration in seconds (Dua Lipa - Levitating): {lib.get_duration(dua_lipa, sr=sr, n_fft=frame, hop_length=hop)}')

# Comparing to Spotify measured duration
print(f"Spotify measured duration in seconds (Dua Lipa - Levitating): {df['duration_s'][55871]}")
print('-------------------------------------------------------------')
print('')

# Measuring duration in seconds for Joji - Attention using Librosa
print(f'Librosa measured duration in seconds (Joji - Attention): {lib.get_duration(joji_attn, sr=sr, n_fft=frame, hop_length=hop)}')

# Comparing to Spotify measured duration
print(f"Spotify measured duration in seconds (Joji - Attention): {df['duration_s'][51917]}")
print('-------------------------------------------------------------')
print('')

# Measuring duration in seconds for James Blake - Retrograde using Librosa
print(f'Librosa measured duration in seconds (Joji - Attention): {lib.get_duration(james_blake, sr=sr, n_fft=frame, hop_length=hop)}')

# Comparing to Spotify measured duration
print(f"Spotify measured duration in seconds (Joji - Attention): {df['duration_s'][43245]}")
print('-------------------------------------------------------------')
print('')

# Measuring duration in seconds for Joey Bada$$ - Paper Trails using Librosa
print(f'Librosa measured duration in seconds (Joji - Attention): {lib.get_duration(joey, sr=sr, n_fft=frame, hop_length=hop)}')

# Comparing to Spotify measured duration
print(f"Spotify measured duration in seconds (Joji - Attention): {df['duration_s'][46528]}")
print('-------------------------------------------------------------')
print('')

---
**BELOW:**

In [ ]:
# Measuring tempo in BPM for Avicii - Levels using Librosa
print(f'Librosa measured tempo in BPM (Avicii - Levels): {lib.beat.tempo(avicii, sr=sr, hop_length=hop, aggregate=np.mean)}')

# Comparing to Spotify measured tempo
print(f"Spotify measured tempo in BPM (Avicii - Levels): {df['tempo'][40245]}")
print('-------------------------------------------------------------')
print('')

# Measuring tempo in BPM for Dua Lipa - Levitating using Librosa
print(f'Librosa measured tempo in BPM (Dua Lipa - Levitating): {lib.beat.tempo(dua_lipa, sr=sr, hop_length=hop, aggregate=np.mean)}')

# Comparing to Spotify measured tempo
print(f"Spotify measured tempo in BPM (Dua Lipa - Levitating): {df['tempo'][55871]}")
print('-------------------------------------------------------------')
print('')

# Measuring tempo in BPM for Joji - Attention using Librosa
print(f'Librosa measured tempo in BPM (Joji - Attention): {lib.beat.tempo(joji_attn, sr=sr, hop_length=hop, aggregate=np.mean)}')

# Comparing to Spotify measured tempo
print(f"Spotify measured tempo in BPM (Joji - Attention): {df['tempo'][51917]}")
print('-------------------------------------------------------------')
print('')

# Measuring tempo in BPM for James Blake - Retrograde using Librosa
print(f'Librosa measured tempo in BPM (James Blake - Retrograde): {lib.beat.tempo(james_blake, sr=sr, hop_length=hop, start_bpm=75, aggregate=np.mean)}')

# Comparing to Spotify measured tempo
print(f"Spotify measured tempo in BPM (James Blake - Retrograde): {df['tempo'][43245]}")
print('-------------------------------------------------------------')
print('')

# Measuring tempo in BPM for Joey Bada$$ - Paper Trails using Librosa
print(f'Librosa measured tempo in BPM (Joey Bada$$ - Paper Trails): {lib.beat.tempo(joey, sr=sr, hop_length=hop, start_bpm=90, aggregate=np.mean)}')

# Comparing to Spotify measured tempo
print(f"Spotify measured tempo in BPM (Joey Bada$$ - Paper Trails): {df['tempo'][46528]}")
print('-------------------------------------------------------------')
print('')

**ABOVE:** Spotify's tempo metric for *Retrograde*, by James Blake, is significantly more accurate than Librosa's measurement in this case. I had to introduce a parameter into Librosa's `tempo` algorithm called `start_bpm`, which is an estimate by the user, and gives Librosa a reference point to start beat tracking. Without this parameter, Librosa was predicting a tempo of about 123bpm.

In [ ]:
tempo_james, beats1 = lib.beat.beat_track(james_blake, sr=sr, hop_length=hop, units='time')

In [ ]:
tempo_james * 2

In [ ]:
tempo_joey, beats2 = lib.beat.beat_track(joey, sr=sr, hop_length=hop, units='time')

In [ ]:
tempo_joey

In [ ]:
# pitches, magnitudes = lib.piptrack(avicii, sr=sr, n_fft=frame, hop_length=hop)